# Cost subsystem 

### Optimisation

This code will allow you to understand and perform an optimisation on the cost subsystem. The code below is a shortened version of the full code available in thisrepository. The aim of this tutorial is not to demostrate the full potential of this code but to take you thorugh the analysis leading to the solutions for the cost subsystem presented in the report.

In [2]:
import time
import numpy as np
import numpy.ma as ma
import operator as op
import matplotlib.pyplot as plt
from scipy.optimize import minimize, BFGS, LinearConstraint, NonlinearConstraint

Multiple classes were used in common with the light subsytem. This allowed an easier integrationand development of both subsytems for the system level analysis.

The most improtant is MP, which stands for the model parameters. This includes the parameters and constrains- presented in a matrix form - for the optimisation problems. For the cost subsystem specifically, a function containing the non linear constrain is also defined. Centralising the common parameters and constrains allowed to make changes without affecting the functions or model themselves. This was a logic step to ease the future implementation into an app where the variables will be constantly be changing. 

In [3]:
class MP:
    """
    Enum to hold some of the Model Parameters
    """

    """ 
    Global Parameters
    """

    # Discretisation step
    DXY = 0.01

    # Room geometry
    ROOM_LENGTH = 4
    ROOM_WIDTH = 3
    ROOM_HEIGHT = 2.3
    F_PLUG_POSITION = [2.3, 0.05]
    S_PLUG_POSITION = [3.95, 2]


    # Number of lamps
    N_LAMPS = 3

    # Parameters
    LAMP_EFFICIENCY = 0.8
    LAMP_RADII = [0.1, 0.2, 0.1]

    POWER_SCALING_FACTOR = 1
    LAMP_POW = [50, 120, 50]
    LAMP_POW = np.array(LAMP_POW) * POWER_SCALING_FACTOR

    # Albedo
    ALBEDO = 0.5
    BOUNCES = 3

    # Plot parameters
    N_LEVELS = 20
    
    """
    Cost Subsystem
    """

    # Cost
    CABLE_COST = 2
    WORK_COST = 40
    ENERGY_COST = 0.12
    AVG_HOURS_PER_YEAR = float(2500 / 1000)
    INVESTMENT_FACTOR = 3
    # Initial characteristics for lamps
    INITIAL_SOLUTION = np.array([0.68978269, 0.98767149, 1.78447148, 2.79305784, 3.66072114, 2.22234, 0.2])
     
     # Lamp 1: Bed: Bound Constraints (x1, y1)
    G1 = [LAMP_RADII[0], 2.3 - LAMP_RADII[0]]
    G2 = [LAMP_RADII[0], 1.5 - LAMP_RADII[0]]

    # Lamp 2: Floor: Bound Constraints (x2, y2)
    G3 = [0.4 + LAMP_RADII[1], 2.3 - LAMP_RADII[1]]
    G4 = [0.9 + LAMP_RADII[1], 3 - LAMP_RADII[1]]

    # Lamp 3: Desk: Bound Constraints (x3, y3)
    G5 = [2.3 + LAMP_RADII[2], 4 - LAMP_RADII[2]]
    G6 = [1.1 + LAMP_RADII[2], 3 - LAMP_RADII[2]]

    # Efficiency, in a range from 0 to 1
    G7 = [0.2, 1]

    CONSTRAINT_MAT_EXT = [[1, 0, 0, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0, 0, 0],
                          [0, 0, 1, 0, 0, 0, 0],
                          [0, 0, 0, 1, 0, 0, 0],
                          [0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 0, 0, 1, 0],
                          [0, 0, 0, 0, 0, 0, 1]]

    # Add all constrains to limit all variables to 0

    CONSTRAINTS_EXT = [G1, G2, G3, G4, G5, G6, G7]

    # Linear Constraint Bounds
    LOWER_BOUND_EXT = [constraint[0] for constraint in CONSTRAINTS_EXT]
    UPPER_BOUND_EXT = [constraint[1] for constraint in CONSTRAINTS_EXT]


"""
FUNCTIONAL CONSTRAINTS
"""

def functional_constraint(variables):
    c_cable_tot = 0
    total_power = sum(MP.LAMP_POW)

    for i in range(3):
        c_cable_tot += (abs(variables[2 * i]) + abs(variables[2 * i + 1])) + 1.6
        c_cable_tot = c_cable_tot * MP.CABLE_COST

    lamp_efficiency = variables[6]
    c_operation = (total_power / lamp_efficiency) * MP.AVG_HOURS_PER_YEAR * MP.ENERGY_COST
    c_lamp = (lamp_efficiency / 0.2)
    c_tot_lamp_cost = MP.N_LAMPS * c_lamp
    c_work = np.log(MP.N_LAMPS) * MP.WORK_COST
    c_initial = (c_cable_tot + c_tot_lamp_cost + c_work)

    return c_initial - MP.INVESTMENT_FACTOR * c_operation

The code below defines the cost funtion for cost of lightning a room. The function calculatesinitial and operation costs, which combined constitute the total cost. The variables are the x and y positions for three different lamp and the common efficiency of these lamps.   

In [4]:
def cost_obj_fun(variables):
    """
    Cost objective function. Vars is [x1, y1, x2, y2, x3, y3, e, i]
    Where c is equal for the three lamps and is equal to the the characteristics of lamp composed by price and efficiency
    """
    #Total cable cost is initialised
    c_cable_tot = 0

    # The power is fixed, so the total power is a sum of the power of all the lamps
    total_power = sum(MP.LAMP_POW)

    '''
    There are 7 variables in total
    In a range of 3 - which equals for iterations,  i = 0,1 and 2 will refer to each of the lamps. 
    The code will consider which considers i and i+1 for x, y  
    '''
    #Start the loop to calculate cable length
    for i in range(3):
        #Calcilate total length, including 1.6 m for the height of the lamps in the room.
        c_cable_tot += (abs(variables[2 * i]) + abs(variables[2 * i + 1])) + 1.6
        #Calculate the total price
        c_cable_tot = c_cable_tot * MP.CABLE_COST

    lamp_efficiency = variables[6]
    # Calculate the operation cost in terms of the efficiency
    c_operation = (total_power/lamp_efficiency) * MP.AVG_HOURS_PER_YEAR * MP.ENERGY_COST

    #Calculate the lamp cost in terms of the efficiency
    c_lamp = (lamp_efficiency/0.2)
    
    # Calculate total lamp cost
    c_tot_lamp_cost = MP.N_LAMPS * c_lamp
    
    # Calculate work cost 
    c_work = np.log(MP.N_LAMPS) * MP.WORK_COST
    
    #Caltulate the initial cost 
    c_initial = (c_cable_tot + c_tot_lamp_cost + c_work)
    
    #Caltulate total cost 
    c_tot = c_initial + c_operation

    return c_tot




The following code presents the trust region algorithm "trust- constr", which is aprt of the Scypy library. It takes into account the linear and non linear constrains defined in model parameters. An initial guess is defined and the model optimises it. The recommendeed hessian is defined as BFGS. 

In [ ]:
class TrustConstrModel:
    """
    Two-dimensional model of light distribution in a plane with n number of light sources
    """

    def __init__(self):

        # To keep track of the iterations
        self.counter = 0

        # Parameters
        self.name = 'Trust-Constr'
        self.refl = False
        self.save_fig = False
        self.save_log = False
        self.constrained = True

        if self.save_log:
            self.data = []

        if self.constrained:
            self.constraints = (LinearConstraint(MP.CONSTRAINT_MAT_EXT, MP.LOWER_BOUND_EXT, MP.UPPER_BOUND_EXT),
                                NonlinearConstraint(functional_constraint, -np.inf, 0, jac='cs', hess=BFGS()))
        else:
            self.constraints = ()

        print("You are using a trust-constr optimiser.")
        print("Constraints: ", self.constrained)

        time.sleep(1)
        # Objective function. We want to maximise this
        self.result = minimize(self.obj_fun, MP.INITIAL_SOLUTION, method='trust-constr', jac='3-point',
                               constraints=self.constraints, hess=BFGS(exception_strategy='damp_update'))

        # What is the result of the optimisation?
        print(self.result)

    def obj_fun(self, variables):
        """
        Cost objective function. Vars is [x1, y1, x2, y2, x3, y3, c]
        Where c is equal for the three lamps and is equal to the the characteristics
        of lamp composed by price and efficiency
        Call the function cost
        """
        # Calculate current cost
        c_tot = cost_obj_fun(variables)

        print("Iteration: ", self.counter, "Positions and efficiency", variables, "total cost", c_tot)
        self.counter += 1
        return c_tot


if __name__ == '__main__':

    model = TrustConstrModel()
    PlotTestDistribution(model.result.x, model.name, refl=model.refl, save_fig=model.save_fig, fig_name=model.name,
                         constrained=model.constrained, cost_subsystem=True)

In [6]:
model = TrustConstrModel()

You are using a trust-constr optimiser.
Constraints:  True


Iteration:  0 Positions and efficiency [0.68978269 0.98767149 1.78447148 2.79305784 3.66072114 2.22234
 0.2       ] total cost 442.8403645467244
Iteration:  1 Positions and efficiency [0.68977663 0.98767149 1.78447148 2.79305784 3.66072114 2.22234
 0.2       ] total cost 442.84031610308875
Iteration:  2 Positions and efficiency [0.68978875 0.98767149 1.78447148 2.79305784 3.66072114 2.22234
 0.2       ] total cost 442.84041299036
Iteration:  3 Positions and efficiency [0.68978269 0.98766543 1.78447148 2.79305784 3.66072114 2.22234
 0.2       ] total cost 442.84031610308875
Iteration:  4 Positions and efficiency [0.68978269 0.98767755 1.78447148 2.79305784 3.66072114 2.22234
 0.2       ] total cost 442.84041299036
Iteration:  5 Positions and efficiency [0.68978269 0.98767149 1.78446067 2.79305784 3.66072114 2.22234
 0.2       ] total cost 442.8403213235813
Iteration:  6 Positions and efficiency [0.68978269 0.98767149 1.78448229 2.79305784 3.66072114 2.22234
 0.2       ] total cost 442.8

Iteration:  150 Positions and efficiency [0.72741112 0.91990052 1.6876786  2.61753326 3.54892249 2.19999411
 0.57111206] total cost 232.37235720949343
Iteration:  151 Positions and efficiency [0.72740507 0.91990052 1.6876786  2.61753326 3.54892249 2.19999411
 0.57111206] total cost 232.37230876585784
Iteration:  152 Positions and efficiency [0.72741718 0.91990052 1.6876786  2.61753326 3.54892249 2.19999411
 0.57111206] total cost 232.37240565312908
Iteration:  153 Positions and efficiency [0.72741112 0.91989446 1.6876786  2.61753326 3.54892249 2.19999411
 0.57111206] total cost 232.37230876585784
Iteration:  154 Positions and efficiency [0.72741112 0.91990657 1.6876786  2.61753326 3.54892249 2.19999411
 0.57111206] total cost 232.37240565312908
Iteration:  155 Positions and efficiency [0.72741112 0.91990052 1.68766838 2.61753326 3.54892249 2.19999411
 0.57111206] total cost 232.3723163308498
Iteration:  156 Positions and efficiency [0.72741112 0.91990052 1.68768882 2.61753326 3.5489224

total cost 217.94919889838678
Iteration:  240 Positions and efficiency [0.72202325 0.91436267 1.68491003 2.61485717 3.54713562 2.19890467
 0.67584252] total cost 215.9202518561233
Iteration:  241 Positions and efficiency [0.72201719 0.91436267 1.68491003 2.61485717 3.54713562 2.19890467
 0.67584252] total cost 215.9202034124877
Iteration:  242 Positions and efficiency [0.72202931 0.91436267 1.68491003 2.61485717 3.54713562 2.19890467
 0.67584252] total cost 215.92030029975894
Iteration:  243 Positions and efficiency [0.72202325 0.91435661 1.68491003 2.61485717 3.54713562 2.19890467
 0.67584252] total cost 215.9202034124877
Iteration:  244 Positions and efficiency [0.72202325 0.91436872 1.68491003 2.61485717 3.54713562 2.19890467
 0.67584252] total cost 215.92030029975894
Iteration:  245 Positions and efficiency [0.72202325 0.91436267 1.68489983 2.61485717 3.54713562 2.19890467
 0.67584252] total cost 215.9202110445395
Iteration:  246 Positions and efficiency [0.72202325 0.91436267 1.68

 204.62472595139397
Iteration:  374 Positions and efficiency [0.71464676 0.90682589 1.68102107 2.61072022 3.54453446 2.197447
 0.77637193] total cost 204.62358147983412
Iteration:  375 Positions and efficiency [0.7137338  0.9058619  1.68054162 2.61010596 3.54423094 2.19720559
 0.78698935] total cost 203.61546537942013
Iteration:  376 Positions and efficiency [0.71372774 0.9058619  1.68054162 2.61010596 3.54423094 2.19720559
 0.78698935] total cost 203.6154169357845
Iteration:  377 Positions and efficiency [0.71373986 0.9058619  1.68054162 2.61010596 3.54423094 2.19720559
 0.78698935] total cost 203.61551382305575
Iteration:  378 Positions and efficiency [0.7137338  0.90585584 1.68054162 2.61010596 3.54423094 2.19720559
 0.78698935] total cost 203.6154169357845
Iteration:  379 Positions and efficiency [0.7137338  0.90586795 1.68054162 2.61010596 3.54423094 2.19720559
 0.78698935] total cost 203.61551382305575
Iteration:  380 Positions and efficiency [0.7137338  0.9058619  1.68053144 2.6

...

Iteration:  14355 Positions and efficiency [0.11923142 0.12826865 1.22241941 2.13942908 3.29953784 1.97080332
 0.99773731] total cost 173.42830422832287
Iteration:  14356 Positions and efficiency [0.11922537 0.12826865 1.22241941 2.13942908 3.29953784 1.97080332
 0.99773731] total cost 173.42825578468728
Iteration:  14357 Positions and efficiency [0.11923748 0.12826865 1.22241941 2.13942908 3.29953784 1.97080332
 0.99773731] total cost 173.42835267195852
Iteration:  14358 Positions and efficiency [0.11923142 0.12826259 1.22241941 2.13942908 3.29953784 1.97080332
 0.99773731] total cost 173.42825578468728
Iteration:  14359 Positions and efficiency [0.11923142 0.1282747  1.22241941 2.13942908 3.29953784 1.97080332
 0.99773731] total cost 173.42835267195852
Iteration:  14360 Positions and efficiency [0.11923142 0.12826865 1.22241201 2.13942908 3.29953784 1.97080332
 0.99773731] total cost 173.42827461910255
Iteration:  14361 Positions and efficiency [0.11923142 0.12826865 1.22242682 2.139

Iteration:  14535 Positions and efficiency [0.11921599 0.12821564 1.21652    2.1335504  3.29647032 1.96808163
 0.99773784] total cost 173.3690386150585
Iteration:  14536 Positions and efficiency [0.11920993 0.12821564 1.21652    2.1335504  3.29647032 1.96808163
 0.99773784] total cost 173.36899017142287
Iteration:  14537 Positions and efficiency [0.11922205 0.12821564 1.21652    2.1335504  3.29647032 1.96808163
 0.99773784] total cost 173.3690870586941
Iteration:  14538 Positions and efficiency [0.11921599 0.12820959 1.21652    2.1335504  3.29647032 1.96808163
 0.99773784] total cost 173.36899017142287
Iteration:  14539 Positions and efficiency [0.11921599 0.1282217  1.21652    2.1335504  3.29647032 1.96808163
 0.99773784] total cost 173.3690870586941
Iteration:  14540 Positions and efficiency [0.11921599 0.12821564 1.21651264 2.1335504  3.29647032 1.96808163
 0.99773784] total cost 173.36900914873263
Iteration:  14541 Positions and efficiency [0.11921599 0.12821564 1.21652737 2.133550

Iteration:  14715 Positions and efficiency [0.11920068 0.12816318 1.21062479 2.12767985 3.29340506 1.96534811
 0.99773837] total cost 173.30980872510065
Iteration:  14716 Positions and efficiency [0.11919463 0.12816318 1.21062479 2.12767985 3.29340506 1.96534811
 0.99773837] total cost 173.30976028146503
Iteration:  14717 Positions and efficiency [0.11920674 0.12816318 1.21062479 2.12767985 3.29340506 1.96534811
 0.99773837] total cost 173.30985716873627
Iteration:  14718 Positions and efficiency [0.11920068 0.12815713 1.21062479 2.12767985 3.29340506 1.96534811
 0.99773837] total cost 173.30976028146503
Iteration:  14719 Positions and efficiency [0.11920068 0.12816924 1.21062479 2.12767985 3.29340506 1.96534811
 0.99773837] total cost 173.30985716873627
Iteration:  14720 Positions and efficiency [0.11920068 0.12816318 1.21061746 2.12767985 3.29340506 1.96534811
 0.99773837] total cost 173.30977940156748
Iteration:  14721 Positions and efficiency [0.11920068 0.12816318 1.21063212 2.127

Iteration:  14865 Positions and efficiency [0.11918802 0.12811986 1.20571604 2.12279326 3.29085216 1.96306136
 0.99773881] total cost 173.26047772093966
Iteration:  14866 Positions and efficiency [0.11918197 0.12811986 1.20571604 2.12279326 3.29085216 1.96306136
 0.99773881] total cost 173.26042927730404
Iteration:  14867 Positions and efficiency [0.11919408 0.12811986 1.20571604 2.12279326 3.29085216 1.96306136
 0.99773881] total cost 173.26052616457528
Iteration:  14868 Positions and efficiency [0.11918802 0.12811381 1.20571604 2.12279326 3.29085216 1.96306136
 0.99773881] total cost 173.26042927730404
Iteration:  14869 Positions and efficiency [0.11918802 0.12812592 1.20571604 2.12279326 3.29085216 1.96306136
 0.99773881] total cost 173.26052616457528
Iteration:  14870 Positions and efficiency [0.11918802 0.12811986 1.20570874 2.12279326 3.29085216 1.96306136
 0.99773881] total cost 173.26044851630536
Iteration:  14871 Positions and efficiency [0.11918802 0.12811986 1.20572334 2.122

 14985 Positions and efficiency [0.11917795 0.12808546 1.20179196 2.11888728 3.28881062 1.96122638
 0.99773916] total cost 173.22103069340727
Iteration:  14986 Positions and efficiency [0.11917189 0.12808546 1.20179196 2.11888728 3.28881062 1.96122638
 0.99773916] total cost 173.22098224977165
Iteration:  14987 Positions and efficiency [0.11918401 0.12808546 1.20179196 2.11888728 3.28881062 1.96122638
 0.99773916] total cost 173.2210791370429
Iteration:  14988 Positions and efficiency [0.11917795 0.1280794  1.20179196 2.11888728 3.28881062 1.96122638
 0.99773916] total cost 173.22098224977165
Iteration:  14989 Positions and efficiency [0.11917795 0.12809151 1.20179196 2.11888728 3.28881062 1.96122638
 0.99773916] total cost 173.2210791370429
Iteration:  14990 Positions and efficiency [0.11917795 0.12808546 1.20178468 2.11888728 3.28881062 1.96122638
 0.99773916] total cost 173.22100158382133
Iteration:  14991 Positions and efficiency [0.11917795 0.12808546 1.20179924 2.11888728 3.28881

Once the optimised solution had been obtained and the minimiser has returned a True solution, this is plotted in a visual way. In order to plot the cables plot distribution function is used. In this function the cables are plotted connecting them to the nearest plug. The cost subsystem integrates the cable visualisation into the light subsystem's visualisation, in order for the the cables to also be connected to the lights. 

To be able to plot the lights with the variable position given by the cost function, the instensity distribution function and room class need to be run first.

In [7]:
def get_intensity_distr(lamp_locs, refl=False):
    """
    Calculates the intensity distribution within a room with n number of light sources
    """

    room = Room(MP.ROOM_LENGTH, MP.ROOM_WIDTH)

    # No reflections
    if not refl:

        for i in range(MP.N_LAMPS):
            # Multiply by discretisation step to be in metres
            distance_to_lamp_n = ((room.xx - lamp_locs[2 * i]) ** 2 + (room.yy - lamp_locs[2 * i + 1]) ** 2)

            # Take out the value that are less than a radius away from the light source
            distance_to_lamp_n_filtered = ma.masked_less(np.sqrt(distance_to_lamp_n), MP.LAMP_RADII[i])

            if i == 0:
                # Initialise the light intensity array
                light_intensity = np.zeros_like(distance_to_lamp_n_filtered)

            # Find light intensity distribution
            light_intensity_n = ((MP.LAMP_EFFICIENCY * MP.LAMP_POW[i]) /
                                 (4 * np.pi)) / (distance_to_lamp_n_filtered * MP.DXY)

            light_intensity += light_intensity_n

        minimum = np.amin(light_intensity)
        minimum_coordinates = np.unravel_index(np.argmin(light_intensity), light_intensity.shape)

        return light_intensity, minimum, minimum_coordinates

    # With reflections
    elif refl:

        initialised = False

        # The first loop takes care of wall reflections
        for i in range(- MP.BOUNCES // 2, MP.BOUNCES // 2 + 1):
            # The second loop takes care of the three different lamps
            for j in range(MP.N_LAMPS):

                x_jk = 0.5 * (1 + (-1) ** i) * lamp_locs[2 * j] + \
                       0.5 * (1 - (-1) ** i) * (MP.ROOM_LENGTH - lamp_locs[2 * j])
                y_jk = 0.5 * (1 + (-1) ** i) * lamp_locs[2 * j + 1] + \
                       0.5 * (1 - (-1) ** i) * (MP.ROOM_WIDTH - lamp_locs[2 * j + 1])

                distance_to_lamp_n = ((room.xx - (x_jk - MP.ROOM_LENGTH * i)) ** 2 +
                                      (room.yy - (y_jk - MP.ROOM_WIDTH * i)) ** 2)

                # Take out the value that are less than a radius away from the light source
                distance_to_lamp_n_filtered = ma.masked_less(np.sqrt(distance_to_lamp_n), MP.LAMP_RADII[j])

                # Initialise the light intensity array
                if not initialised:
                    light_intensity = np.zeros_like(distance_to_lamp_n_filtered)
                    initialised = True

                # Find light intensity distribution
                light_intensity_n = ((MP.ALBEDO ** (abs(2 * i))) * (MP.LAMP_EFFICIENCY * MP.LAMP_POW[j]) /
                                     (4 * np.pi)) / (distance_to_lamp_n_filtered * MP.DXY)

                # Increment light intensity array
                light_intensity += light_intensity_n

        minimum = np.amin(light_intensity)
        minimum_coordinates = np.unravel_index(np.argmin(light_intensity), light_intensity.shape)

        return light_intensity, minimum, minimum_coordinates

class Room:
    """
    The Room class contains the space in which the lighting optimisation will take place
    """

    def __init__(self, length, width):

        self.x = np.arange(0, length, MP.DXY)
        self.y = np.arange(0, width, MP.DXY)
        self.xx, self.yy = np.meshgrid(self.x, self.y, sparse=True)


In [1]:
class PlotTestDistribution:
    """
    Two-dimensional model of light distribution in a plane with n number of light sources
    """

    def __init__(self,
                 lamp_locs = (1, 1, 1.5, 2, 3, 2.5),
                 name='',
                 refl=False,
                 save_fig=False,
                 fig_name='',
                 constrained=False,
                 cost_subsystem=False,
                 weight=-1.0):

        # These cannot be taken from the enum, as they vary upon each instatiation
        self.lamp_locs = lamp_locs
        self.name = name
        self.refl = refl
        self.save_fig = save_fig
        self.fig_name = fig_name
        self.constrained = constrained
        self.cost_subsystem = cost_subsystem
        self.weight = weight

        # Define plugs position
        self.firstplug_position = MP.F_PLUG_POSITION
        self.secondplug_position = MP.S_PLUG_POSITION

        # Plot
        if not self.cost_subsystem:
            self.plot_intensity_distr()
        else:
            self.plot_cable_distribution()

    def plot_intensity_distr(self):
        """
        Calculates the intensity distribution within a room with n number of light sources
        """

        light_intensity, minimum, minimum_coordinates = get_intensity_distr(self.lamp_locs, self.refl)

        # Fill masked areas (lamps) with maximum value
        light_intensity = ma.filled(light_intensity, np.amax(light_intensity))

        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)

        # Plot background image of room
        if not self.constrained:
            img = plt.imread("../classes/room_outline.png")
        else:
            img = plt.imread("../classes/room_outline_constr.png")

        ax.imshow(img, extent=[0, MP.ROOM_LENGTH / MP.DXY, 0, MP.ROOM_WIDTH / MP.DXY])

        # Plot intensity distribution
        ax.contourf(light_intensity, MP.N_LEVELS, cmap='plasma', alpha=0.75, antialiased=True)

        # Plot global minimum
        ax.plot(minimum_coordinates[1], minimum_coordinates[0], 'ro', markersize=12)

        # Layout & Titles
        plt.suptitle(self.name + " Optimisation", fontweight='bold')

        plt.title("Reflections: " + str(self.refl) + ", Constraints: " + str(self.constrained)
                  + ", Minimum: " + str(round(minimum, 2)), fontsize='large')

        plt.xlabel('Room X-Position (cm)')
        plt.ylabel('Room Y-Position (cm)')

        # Export figure
        if self.save_fig:
            plt.savefig(self.fig_name + str(self.refl) + str(self.constrained) + '.svg', format='svg', dpi=1200)

        # Show figure

        plt.show()

    def plot_cable_distribution(self):
        """
               Calculates the intensity distribution within a room with n number of light sources
               """

        #light_intensity, minimum, minimum_coordinates = get_intensity_distr(self.lamp_locs, self.refl)
        light_intensity, minimum, minimum_coordinates = get_intensity_distr(self.lamp_locs, self.refl)
        total_cost = cost_obj_fun(self.lamp_locs)

        # Fill masked areas (lamps) with maximum value
        light_intensity = ma.filled(light_intensity, np.amax(light_intensity))

        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)

        # Plot background image of room
        if not self.constrained:
            img = plt.imread("../classes/room_outline.png")
        else:
            img = plt.imread("../classes/room_outline_constr.png")

        ax.imshow(img, extent=[0, MP.ROOM_LENGTH / MP.DXY, 0, MP.ROOM_WIDTH / MP.DXY])

        # Plot intensity distribution
        ax.contourf(light_intensity, MP.N_LEVELS, cmap='plasma', alpha=0.75, antialiased=True)

        # Plot global minimum
        ax.plot(minimum_coordinates[1], minimum_coordinates[0], 'ro', markersize=12)


        # plot cost
        for i in range(MP.N_LAMPS):

            lamp_position = [self.lamp_locs[2 * i], self.lamp_locs[2 * i + 1]]
            # distance to first plug and second plang
            d_fp = list(map(op.sub, lamp_position, self.firstplug_position))
            d_sp = list(map(op.sub, lamp_position, self.secondplug_position))

            # therefore cable length for L shape
            l_fp = (abs(d_fp[0]) + abs(d_fp[1]))
            l_sp = (abs(d_sp[0]) + abs(d_sp[1]))

            #print("cable length to plug 1: ", l_fp)
            #print("cable length to plug 2: ", l_sp)

            # Seems to be x1, x2,y1,y2 - look into this
            if l_fp <= l_sp:
                plt.plot([100 * self.firstplug_position[0], 100 * self.lamp_locs[2 * i]],
                         [100 * self.firstplug_position[1], 100 * self.firstplug_position[1]], "grey")
                plt.plot([100 * self.lamp_locs[2 * i], 100 * self.lamp_locs[2 * i]],
                         [100 * self.firstplug_position[1], 100 * self.lamp_locs[2 * i + 1]], "grey")
            else:
                plt.plot([100 * self.secondplug_position[0], 100 * self.lamp_locs[2 * i]],
                         [100 * self.secondplug_position[1], 100 * self.secondplug_position[1]], "grey")
                plt.plot([100 * self.lamp_locs[2 * i], 100 * self.lamp_locs[2 * i]],
                         [100 * self.secondplug_position[1], 100 * self.lamp_locs[2 * i + 1]], "grey")


        # Layout & Titles
        plt.suptitle(self.name + " Optimisation", fontweight='bold')

        if self.weight <= 0.0:
            plt.title("Constraints: " + str(self.constrained) + ", Minimum: " + str(round(minimum, 2)) +
                      ", Cost: " + str(round(total_cost, 2)), fontsize='large')
        else:
            plt.title("Constraints: " + str(self.constrained) + ", Minimum: " + str(round(minimum, 2)) +
                      ", Cost: " + str(round(total_cost, 2)) + " Weight (Light): " + str(self.weight), fontsize='large')

        plt.xlabel('Room X-Position (cm)')
        plt.ylabel('Room Y-Position (cm)')
        # Export figure
        if self.save_fig:
            plt.savefig('../plots/' + self.fig_name + str(self.refl) + str(self.constrained) + str(self.cost_subsystem) + 
            	'.svg', format='svg', dpi=1200)

        # Show figure
        plt.show()


In [2]:
PlotTestDistribution(model.result.x, model.name, refl=model.refl, save_fig=model.save_fig, fig_name=model.name,
                         constrained=model.constrained, cost_subsystem=True)

NameError: name 'model' is not defined